# Preprocessing

This notebook is used to process the raw transcripts into a format that can then be used for model training and testing.


In [1]:
#!/usr/bin/env/python
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import string
import unicodedata
import matplotlib
import matplotlib.pyplot as plt
import re

In [2]:
#Load Subset 1 and Subset 2
path1 = '../data/Subset1_WithDialogActs/'
subdirs = glob.glob('%s*'%(path1))

path2 = '../data/Subset2_WithDialogActs/'
subdirs.extend(glob.glob('%s*'%(path2)))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [3]:
def resolve_label(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    elif label =='8':
        return '5'
    elif label =='9':
        return '6'
    elif label =='7' or label == '6':
        return 'nan'
    
def resolve_slabel(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    else:
        return 'nan'

In [4]:
'''
To avoid overfitting the models we carefully divide the training and testing set. 
Testing set include some teachers who particiapted in Year 1. The transcripts from these teachers are not included
in the training set. The teachers in this list were picked at random.
'''

train_subdirs =[]
test_subdirs = []
for item in subdirs:
    if item.find("Travers.Spring") != -1 or item.find("Barry.Spring") != -1 \
    or item.find("Carroll") != -1 or item.find("Saunders.Spring") != -1 \
    or item.find("Keene") != -1 or item.find("Carter") != -1 or item.find("Basker") != -1 :
        test_subdirs.append(item)
    else:
        train_subdirs.append(item)
print(len(train_subdirs), len(test_subdirs))

503 63


In [5]:
data = None
temp = []
flag = 0
for dir in train_subdirs: #replace with test_subdirs to generate test data
    if dir.endswith('.xlsx') and not dir.startswith('~'):
        try:
            temp_data = None
            load_data = pd.read_excel(dir, engine='openpyxl')
            load_data['Sentence'] = load_data['Sentence'].str.replace("[\(\[].*?[\)\]]", "", regex=True)
            load_data = load_data[load_data['Sentence'] != ""]
            mask = load_data['Sentence'].str.contains('(.*[\(].*|.*[\)].*|.*[\[].*|.*[\]].*)|(^[.?!]+$)|(^.*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2}.*$)',regex=True,na=False)
            temp_data = load_data[:][~mask]
            temp_data = temp_data.loc[:, ~temp_data.columns.str.contains('^Unnamed')]
            temp_data['Transcript'] = dir.split('/')[len(dir.split('/'))-1]
            for item in temp_data.columns:
                if item not in ['Teacher Tag', 'TimeStamp' ,'Turn','Student Tag','Sentence','Speaker','Transcript','DAMSLTag']:
                    temp_data = temp_data.drop(columns=[item])
            if flag == 0:
                data = temp_data
                flag = 1
            else:
                if len(data.columns) > 8:
                    print(data.columns,dir)
                data = data.append(temp_data, ignore_index=True,sort=False)
        except Exception as e:
            print(e,dir)
            pass 

/home/ec2-user/anaconda3/envs/p37/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [6]:
data[(data['DAMSLTag'].notna()) & (data['DAMSLTag'] != '%')]

,TimeStamp,Turn,Speaker,Sentence,Student Tag,DAMSLTag,Transcript,Teacher Tag
0,NaN,1,T,"My name's Ms. Villarin, I don't know if Ms. Fe...",NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
1,NaN,1,T,"So today, her and I will be teaching a lesson ...",NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
2,NaN,1,T,"So you can treat me the way you treat her, if ...",NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
3,NaN,1,T,Okay?,NaN,b,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
4,NaN,1,T,So today we're going to be doing a really cool...,NaN,sd,7th grade math.xlsx_with_dialog_acts.xlsx,NaN
...,...,...,...,...,...,...,...,...
207798,NaN,290,T,"Okay, clear answers to the questions.",NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,1 - none
207799,NaN,290,T,I'm hoping you all rewrote clear answers to yo...,NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,1 - none
207800,NaN,290,T,You did accurate work.,NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,1 - none
207801,NaN,290,T,You made revisions and I am going to look to s...,NaN,sd,Talkback.Year1.Strathom.Fall.121119.xlsx_with_...,2 - keeping everyone together


In [6]:
data = data[(data['DAMSLTag'].notna()) & (data['DAMSLTag'] != '%')]
#print(data.head(), set(data['DAMSLTag']))
data = data[(data['Teacher Tag'].notna()) ^ (data['Student Tag'].notna())]

In [7]:
dataset = []
bad_sentences = []
bad_speakers = []
num_sentences = []
bad_entries = []
translator = str.maketrans('', '', string.punctuation)
for entry in data.iterrows():
    try:
        if type(entry[1]['Speaker']) == int or entry[1]['Speaker'] == float:
            bad_speakers.append(entry[1]['Speaker'])
            continue
        if repr(type(entry[1]['Sentence'])) == "<type 'datetime.time'>" or repr(type(entry[1]['Sentence'])) == "<type 'datetime.datetime'>" or repr(entry[1]['Speaker']) == 'nan':
            bad_sentences.append(entry[1]['Sentence'])
            continue
    
        if repr(type(entry[1]['Sentence'])) == "<type 'int'>" or repr(type(entry[1]['Sentence'])) == "<type 'float'>":
            num_sentences.append(str(entry[1]['Sentence']))
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
        if repr(type(entry[1]['Sentence'])) == "<type 'str'>":
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
            
        temp = unicodedata.normalize('NFKD', str(entry[1]['Sentence']).translate(translator))
        
        # Turn, Speaker, Sentence, Teacher Tag, Student Tag, Dialog Act tag
        try:
            tag = resolve_label(str(entry[1]['Teacher Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        try:
            stag = resolve_slabel(str(entry[1]['Student Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        
        sent = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in temp.split(" ")])
        
        if str(stag) != 'nan':
            #print([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['DAMSLTag'],entry[1]['Transcript']])
        elif str(tag) != 'nan':
            #print([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['DAMSLTag'],entry[1]['Transcript']])
       
    except Exception as e:
        print(e)
        bad_entries.append([entry[1]['Sentence'], entry[1]['Speaker']])
        continue

In [8]:
df = pd.DataFrame({'Turn':[i[0] for i in dataset], 'Speaker':[i[1] for i in dataset], \
                   'Sentence':[i[2] for i in dataset], 'Tag':[i[3] for i in dataset],\
                   'StudentTag':[i[4] for i in dataset], 'DAMSLTag':[i[5] for i in dataset],
                   'Transcript':[i[6] for i in dataset]})
df = df.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df.head(), set(df['Tag']), set(df['StudentTag'])

(  Turn Speaker                                           Sentence   Tag  \
 0    2       S                                            Mmm hmm  None   
 1    2       S  We were looking at the circumference and area ...  None   
 2    5       S                                               Yeah  None   
 3    7       S  We talked about the area is length times width...  None   
 4    9       S                                         a triangle  None   
 
   StudentTag DAMSLTag                                 Transcript  
 0          0       sd  7th grade math.xlsx_with_dialog_acts.xlsx  
 1          3       sd  7th grade math.xlsx_with_dialog_acts.xlsx  
 2          0        b  7th grade math.xlsx_with_dialog_acts.xlsx  
 3          3       sd  7th grade math.xlsx_with_dialog_acts.xlsx  
 4          3       sd  7th grade math.xlsx_with_dialog_acts.xlsx  ,
 {'0', '1', '2', '3', '4', '5', '6', None},
 {'0', '1', '2', '3', '4', 'nan'})

In [9]:
writer = pd.ExcelWriter('../data/train_data_503_dailog_acts.xlsx') #replace with test_data_63 for test data
df.to_excel(writer)
writer.save()

In [ ]:
'''
Pre-processing 

- All lower cased
- All punctuation removed
- Converted all teachers to T tags
- Converted all students to S tags
- T labels range from  [0-6] - 0 stands for None
- S are labelled None
- Compiled all the data into single excel sheet
- At present there are 91891 sentences in total

Cleaning

- [End of Class], [Class Ends] - Sentence in brackets
- If Sentence contains a timestamp or date time stamp
- If sentence is just . or ! or ? or empty
- Speaker is a number (All of them are nans)
- Speaker is nan


'''

In [10]:
#Prepare Tuples from data - Student
data = pd.read_excel('../data/train_data_503_dailog_acts.xlsx', engine='openpyxl') #replace with test_data_63 to generate test data
empty_previous = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        continue
    if row[1]['Speaker'] == 'S' and flag == 0:
        temp =  row[1]['Sentence']
        flag=1
        sentence_tuple.append([empty_previous,temp,row[1]['DAMSLTag'],row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['DAMSLTag'], row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        temp =  row[1]['Sentence']
        continue
        
df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], \
                   'text_b':[i[1] for i in sentence_tuple], \
                   'damsl_labels':[i[2] for i in sentence_tuple],
                   'labels':[i[3] for i in sentence_tuple]})

print(len(df),set(y))
df.to_csv('../data/train_student_dact.tsv',sep='\t',index=False)   #replace with test_student to generate test data  

42857 {0.0, 1.0, 2.0, 3.0, 4.0}


In [11]:
#Prepare Tuples from data - teacher
data = pd.read_excel('../data/train_data_503_dailog_acts.xlsx', engine='openpyxl') #replace with test_data_63 to generate test data
empty_student = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'S' and flag == 2:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 2
        continue
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        if repr(row[1]['Tag']) == 'nan':
            print(row[1])
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        if repr(row[1]['Tag']) == 'nan':
            print(row[1])
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 2:
        if repr(row[1]['Tag']) == 'nan':
            print(row[1])
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['DAMSLTag'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        flag = 1
        continue

df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple],\
                   'text_b':[i[1] for i in sentence_tuple],\
                   'damsl_labels':[i[2] for i in sentence_tuple],\
                   'labels':[i[3] for i in sentence_tuple]})

print(len(df),set(y))
df.to_csv('../data/train_teacher_dact.tsv',sep='\t',index=False)  #replace with test_teacher to generate test data 

143829 {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0}
